# Phase Transition?

We will see if we can get a phase transition when we distribute $x$ as a sparse Laplacian.

In [1]:
import glob
import os
import cv2
import numpy as np
import tqdm
from matplotlib import pyplot as plt

import torch
import torch.distributions as ds
import random

In [180]:
def sample_laplace(scale, sparsity, num_samples):
    """Returns a sample of laplacian with sparsity and quantized to integers."""
    a = ds.Laplace(torch.zeros(num_samples), torch.ones(num_samples) * scale)
    # Sample and sparsify
    sample = a.sample().numpy()
    for index in range(len(sample)):
        if random.random() < sparsity:
            sample[index] = 0
    # sample = np.round(sample)
    return sample

def sample_gaussian(sparsity, num_samples):
    """Returns a sample of gaussian with sparsity and quantized to integers."""
    a = ds.Normal(torch.zeros(num_samples), torch.ones(num_samples))
    # Sample and sparsify
    sample = a.sample().numpy()
    for index in range(len(sample)):
        if random.random() < sparsity:
            sample[index] = 0
    # sample = np.round(sample)
    return sample


Retrieval of compressed sensing should be the linear programming minimization question:

$$\min \| \hat{x} \|_1  ~ ~ ~ ~ ~ ~ \text{s.t. } A \hat{x} = y$$

We can convert this to the linear program:

$$\min \sum_i^{2N} c_i  ~ ~ ~ ~ ~ ~ \text{s.t. } [A, -A] c = y, ~ ~ c \ge 0$$ 

In [181]:
def solve(A, y, M, N):
    """Use scipy linear programming to do the compressed sensing retrieval"""
    from scipy.optimize import linprog
    Aprime = np.hstack([A, A])
    res = linprog(np.ones(2 * N), A_eq=Aprime, b_eq=y, method='interior-point')  # default bounds are >= 0
    print(res.message)
    return res.x[:N] - res.x[N:]

In [182]:
M = 48
N = 50

In [184]:
A_dist = ds.Normal(torch.zeros(M, N), torch.ones(M, N) * (1.0 / np.sqrt(N)))

A = A_dist.sample().numpy()
# x = sample_laplace(1, 0.5, N)
x = sample_gaussian(0.5, N)
y = np.matmul(A, x)

In [185]:
xhat = solve(A, y, M, N)

The algorithm terminated successfully and determined that the problem is infeasible.


In [190]:
x

array([-0.6880156 ,  0.09673514,  0.51045156,  0.64267814,  0.        ,
        0.        ,  0.49751276,  0.        ,  0.72851604,  0.        ,
        1.3408366 , -1.2541963 ,  1.5012279 ,  0.22293298,  1.8803587 ,
        0.        ,  0.67827165,  0.        , -0.26296946,  0.7015628 ,
        0.8602191 , -0.87063205,  0.        ,  1.3034264 , -0.5041305 ,
       -0.29989025,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        , -0.2947102 ,  0.        , -1.1545197 ,  0.23995249,
        0.4512319 , -0.5031786 ,  0.        , -2.05831   ,  0.        ,
       -0.42663136,  0.        ,  0.        ,  0.        , -1.2557834 ,
       -0.04026556,  0.        ,  0.        ,  0.        , -0.14472713],
      dtype=float32)

In [187]:
xhat

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])